# Part I. ETL Pipeline for Pre-Processing the Files
## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES
### import Python packages

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\o.felce\Desktop\Udacity\Data Engineering\PROJECT 2 NoSQL\Data-Modeling-with-Cassandra


### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


#### The new CSV file <font color=red>**event_datafile_new.csv**</font> contains the following columns:

* artist
* firstName of user
* gender of user
* item number in session
* last name of user
* length of the song
* level (paid or free song)
* location of the user
* sessionId
* song title
* userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster()

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print("Cluster created and connected")
except Exception as e:
    print(e)

Cluster created and connected


#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



## Query 1: 
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4 
## SELECT artist, song, lenght FROM song_info_session WHERE session_id = 338 and item_in_session = 4

* Primary key is session_id
* clustering column is item_in_session 


In [7]:
query_1 = "CREATE TABLE IF NOT EXISTS song_info_session"
query_1 = query_1 + "(session_id INT, item_in_session INT, artist TEXT, song TEXT, lenght FLOAT, PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query_1)
    print("Table created")
except Exception as e:
    print(e)

Table created


In [8]:
# Set up CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line) => ["Des'ree", 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', '139', 'You Gotta Be', '8']
        query = "INSERT INTO song_info_session (session_id, item_in_session, artist, song, lenght)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

### Verifying that the data have been inserted into each table

In [10]:
query = "SELECT artist, song, lenght FROM song_info_session WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', lenght=495.30731201171875)


## Query 2: 

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

## SELECT artist, song, user_name FROM song_info_user where sessionId = 182 and userId = 10

* Primary key is sessionId and userId
* clustering column is itemInSession 

In [13]:
query_2 = "CREATE TABLE IF NOT EXISTS song_info_user"
query_2 = query_2 + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, user_name TEXT, PRIMARY KEY(user_id, session_id), itemInSession) WITH CLUSTERING ORDER BY (itemInSession ASC)"

try:
    session.execute(query_1)
    print("Table created")
except Exception as e:
    print(e)

Table created


In [14]:
# Set up CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_user (user_id, session_id, item_in_session, artist, song, user_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + " " + line[4]))

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table song_info_user"

### Drop the tables before closing out the sessions

In [27]:
query = "drop table song_info_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_info_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table song_info_user"


### Close the session and cluster connection

In [28]:
session.shutdown()
cluster.shutdown()